In [11]:
import sqlite3
import pandas as pd

from nltk.stem.snowball import HungarianStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.externals import joblib

In [14]:
# load articles from DB
conn = sqlite3.connect("origo_articles_2014_2018.db")

origo_data = pd.read_sql_query("SELECT article_content, article_title, author, date from origo_data_all", conn)

In [32]:
origo_data['date'] = pd.to_datetime(origo_data['date'])

In [33]:
# extract year, month, day, weekday info from publish date
origo_data['year'] = origo_data['date'].apply(lambda x: int(x.year))
origo_data['month'] = origo_data['date'].apply(lambda x: int(x.month))
origo_data['day'] = origo_data['date'].apply(lambda x: int(x.day))
origo_data['weekday'] = origo_data['date'].apply(lambda x: 1 if x.dayofweek in [0, 1, 2, 3, 4] else 0)

In [34]:
# isOrigo flag: true if article was published anonymously, as Origo
origo_data['author_isOrigo'] = origo_data['author'].apply(lambda x: 1 if x == 'Origo' else 0)

In [35]:
# tokenize article title and content
origo_data['article_content_tokenized'] = origo_data['article_content'].apply(lambda x: word_tokenize(x))
origo_data['article_title_tokenized'] = origo_data['article_title'].apply(lambda x: word_tokenize(x))

In [36]:
# stem tokenized words and remove stopwords
stemmer = HungarianStemmer()
hungarian_stops = set(stopwords.words('hungarian'))
origo_data['article_content_stemmed'] = origo_data['article_content_tokenized'].apply(lambda x: [stemmer.stem(word) for word in x if word not in hungarian_stops])
origo_data['article_title_stemmed'] = origo_data['article_title_tokenized'].apply(lambda x: [stemmer.stem(word) for word in x if word not in hungarian_stops])

In [38]:
origo_data.head(10)

,article_content,article_title,author,date,year,month,day,weekday,author_isOrigo,article_content_tokenized,article_title_tokenized,article_content_stemmed,article_title_stemmed
0,Most olyan tanárok is főiskolai végzettségért ...,Főiskolai fizetést kapnak egyetemi végzettségű...,Origo,2014-04-01 08:52:00,2014,4,1,1,1,"[Most, olyan, tanárok, is, főiskolai, végzetts...","[Főiskolai, fizetést, kapnak, egyetemi, végzet...","[mos, tanár, is, főiskol, végzettség, járó, fi...","[főiskol, fizetés, kap, egyetem, végzettségű, ..."
1,Vádat emeltek a Király Tamást megfojtó 39 éves...,Tizenöt évet is kaphat Király Tamás gyilkosa,-,2014-04-01 09:03:00,2014,4,1,1,0,"[Vádat, emeltek, a, Király, Tamást, megfojtó, ...","[Tizenöt, évet, is, kaphat, Király, Tamás, gyi...","[vád, emelt, király, tamás, megfojtó, 39, éves...","[tizen, év, is, kaph, király, tamás, gyilkos]"
2,Robog tovább a miniszterelnök rózsaszín kanapé...,Orbán Viktor búskomoran ül a rózsaszín kanapéján,Origo,2014-04-01 09:46:00,2014,4,1,1,1,"[Robog, tovább, a, miniszterelnök, rózsaszín, ...","[Orbán, Viktor, búskomoran, ül, a, rózsaszín, ...","[robog, minisztereln, rózsaszí, kanapés, kampá...","[orb, viktor, búskomor, ül, rózsaszí, kanapé]"
3,A köztévéhez köthető hűtlen kezelés miatt nyom...,Közeleg a vádemelés a köztévés fantomcégek ügy...,Origo,2014-04-01 09:21:00,2014,4,1,1,1,"[A, köztévéhez, köthető, hűtlen, kezelés, miat...","[Közeleg, a, vádemelés, a, köztévés, fantomcég...","[a, köztév, köthető, hűtlen, kezelés, miat, ny...","[közeleg, vádemelés, köztévés, fantomcég, ügy]"
4,"Nagyjából 30 ezer levélszavazat veszhet kárba,...",Minden negyedik levélszavazat érvénytelen,MTI,2014-04-01 13:27:00,2014,4,1,1,0,"[Nagyjából, 30, ezer, levélszavazat, veszhet, ...","[Minden, negyedik, levélszavazat, érvénytelen]","[nagy, 30, ezer, levélszavaz, veszh, kár, ,, h...","[min, negyed, levélszavaz, érvénytel]"
5,A PM politikusa szerint egy közteres furgonnal...,Őrséget szerveznek Karácsony Gergely plakátjai...,MTI,2014-04-01 11:30:00,2014,4,1,1,0,"[A, PM, politikusa, szerint, egy, közteres, fu...","[Őrséget, szerveznek, Karácsony, Gergely, plak...","[a, pm, politikus, közteres, furg, utazt, ,, é...","[őrség, szervez, karácsony, gergely, plakát, mel]"
6,Egy férfi életveszélyesen megfenyegetett fides...,Fidesz-aktivistákra támadtak Budapesten,Origo,2014-04-01 13:35:00,2014,4,1,1,1,"[Egy, férfi, életveszélyesen, megfenyegetett, ...","[Fidesz-aktivistákra, támadtak, Budapesten]","[egy, férf, életveszélyes, megfenyegetet, fide...","[fidesz-aktivista, támadt, budapest]"
7,Juhász Péter és Kerék-Bárczy Szabolcs számítás...,"Együtt-PM: Rogán többet költ egy hónapban, min...",Origo,2014-04-01 14:12:00,2014,4,1,1,1,"[Juhász, Péter, és, Kerék-Bárczy, Szabolcs, sz...","[Együtt-PM, :, Rogán, többet, költ, egy, hónap...","[juhász, péter, kerék-bárczy, szabolcs, számít...","[együtt-p, :, rog, több, köl, hónap, ,, amenny..."
8,A 64 kilós példányt csak fél óráig kellett fár...,"Kétméteres harcsát fogott, majd visszadobta",Origo,2014-04-01 14:13:00,2014,4,1,1,1,"[A, 64, kilós, példányt, csak, fél, óráig, kel...","[Kétméteres, harcsát, fogott, ,, majd, visszad...","[a, 64, kilós, példány, fél, ór, fárasztan, .,...","[kétméteres, harcs, fogot, ,, visszadobt]"
9,A matematika után újabb területen égtek le a m...,Megint leégtek a magyar diákok,Origo,2014-04-01 15:17:00,2014,4,1,1,1,"[A, matematika, után, újabb, területen, égtek,...","[Megint, leégtek, a, magyar, diákok]","[a, matemat, terület, égt, le, magyar, diák, ,...","[megin, leégt, magyar, diák]"


In [43]:
# save to DB
df_to_db = origo_data.astype(str)
df_to_db.to_sql('origo_data_enhanced', conn, if_exists = 'replace')